In [ ]:
from HelmholtzSolver3D import *
from ExampleBoundaries import sphere

# Test Problem 1
# Dirichlet boundary condition with phi chose to be the same
# as a point-source at the center of the square.
#
frequency = 100.0 # frequency [Hz]
k = frequency_to_wavenumber(frequency)

solver = ExteriorHelmholtzSolver3D(sphere())

boundary_condition = solver.dirichlet_boundary_condition()
for i in range(solver.centers.shape[0]):
    R = norm(solver.centers[i, :])
    boundary_condition.f[i] = np.exp(1j * k * R) / R

boundary_incidence = BoundaryIncidence(solver.len())
boundary_incidence.phi.fill(0.0)
boundary_incidence.v.fill(0.0)

exterior_points = np.array([[0.0000, 0.0000,  2.0000],
                           [0.0500, 0.0000,  4.0000],
                           [0.1000, 0.0000,  8.0000],
                           [0.0500, 0.1000, -2.0000]], dtype=np.float32)

exterior_incident_phi = np.zeros(exterior_points.shape[0], dtype=np.complex64)

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(exterior_incident_phi, exterior_points)
print("\n\nTest Problem 1")
print("==============\n")
print(boundary_solution)
print(sample_solution)


# Test Problem 2
# Neumann boundary condition with v chosen to be the same
# as a point-source at the center of the square.
#
boundary_condition = solver.neumann_boundary_condition()
centers = solver.geometry.centers()
normals = solver.geometry.normals()
for i in range(solver.centers.shape[0]):
    R = norm(centers[i, :])
    dot = np.dot(solver.centers[i, :], normals[i])
    boundary_condition.f[i] = np.exp(1j * k * R) * dot * (1j * k * R - 1.0) / R**3

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(exterior_incident_phi, exterior_points)
print("\n\nTest Problem 2")
print("==============\n")
print(boundary_solution)
print(sample_solution)


# Test Problem 3
# Dirichlet boundary condition on the full boundary.
# The 400Hz field in this case is generated by a point source
# at (0.5, 0.25)
boundary_condition = solver.dirichlet_boundary_condition()
source_point = np.array([0.0, 0.0, 3.0], dtype=np.float32)
for i in range(solver.centers.shape[0]):
    r = source_point - solver.centers[i, :]
    R = norm(r)
    boundary_incidence.phi[i] = np.exp(1j * k * R) / (4.0 * np.pi * R)
    boundary_condition.f[i] = boundary_incidence.phi[i]
    dot = -np.dot(r, normals[i])
    boundary_incidence.v[i] = np.exp(1j * k * R) * dot * (1j * k * R - 1.0) / (4.0 * np.pi * R**3)

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)

for i in range(exterior_points.shape[0]):
    R = norm(exterior_points[i, :] - source_point)
    exterior_incident_phi[i] = np.exp(1j * k * R) / (4.0 * np.pi * R)
    
sample_solution = boundary_solution.solve_samples(exterior_incident_phi, exterior_points)
print("\n\nTest Problem 3")
print("==============\n")
print(boundary_solution)
print(sample_solution)


# Test Problem 4
# Neumann boundary condition with v=0 on the full boundary.
# The 400Hz field in this case is generated by a point source
# at (0.5, 0.25)
boundary_condition = solver.neumann_boundary_condition()

source_point = np.array([0.0, 0.0, 3.0], dtype=np.float32)
for i in range(solver.centers.shape[0]):
    r = source_point - solver.centers[i, :]
    R = norm(r)
    dot = -np.dot(r, normals[i])
    boundary_incidence.v[i] = np.exp(1j * k * R) * dot * (1j * k * R - 1.0) / (4.0 * np.pi * R**3)

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)

for i in range(exterior_points.shape[0]):
    R = norm(exterior_points[i, :] - source_point)
    exterior_incident_phi[i] = np.exp(1j * k * R) / (4.0 * np.pi * R)
    
sample_solution = boundary_solution.solve_samples(exterior_incident_phi, exterior_points)
print("\n\nTest Problem 4")
print("==============\n")
print(boundary_solution)
print(sample_solution)


Copyright (C) 2017&ndash;2019 Frank Jargstorff

This file is part of the AcousticBEM library.

AcousticBEM is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

AcousticBEM is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with AcousticBEM. If not, see http://www.gnu.org/licenses/.